In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp


In [2]:
# Read weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_df = pd.read_parquet('result_files/us_ca_daily_weather.parquet.gzip', engine="fastparquet") 

In [3]:
# Show weather data column types
weather_data_df.dtypes 

Attr
ID              object
Date    datetime64[ns]
PRCP           float64
TAVG           float64
TMAX           float64
TMIN           float64
dtype: object

In [4]:
# Show weather data
weather_data_df

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
0,CA001011500,2023-01-01,0.0,50.0,80.0,20.0
1,CA001011500,2023-01-02,2.0,50.0,70.0,30.0
2,CA001011500,2023-01-03,12.0,43.0,55.0,30.0
3,CA001011500,2023-01-04,16.0,48.0,65.0,30.0
4,CA001011500,2023-01-05,17.0,70.0,90.0,50.0
...,...,...,...,...,...,...
8862266,USW00096409,2023-12-27,0.0,-205.0,-125.0,-285.0
8862267,USW00096409,2023-12-28,0.0,-177.0,-145.0,-209.0
8862268,USW00096409,2023-12-29,0.0,-220.5,-186.0,-255.0
8862269,USW00096409,2023-12-30,0.0,-247.5,-230.0,-265.0


In [5]:
# Goal: How many days of year in each temperature range

#- Create bins for highs (50-55, 55-6, …. 95-100, 100-105) etc
#- Create bins for lows
#- <or> Alternate: may also try bins for days > 80, days > 90, days < 30 etc.
#- Save results

# Result should look like:
# | Station ID | #days < 10 | #days < 20 | ... | #days < 65 | #days > 60 | ... | #days > 95 | # days > 100 |
# |----|----|----|----|----|----|----|----|----|
# | US001 | 0 | 0 | ... | 300 | 365 | ... | 45 | 20 |
# | US002 | 5 | 15 | ... | 360 | 280 | ... | 5 | 0 |

bins = [-50, -25, -10, 0, 10, 25, 32, 40, 55, 60, 65, 70, 75, 80, 85, 90, 100, 110, 120]
headings = ['-50 To -25', '-25 To -10', '-10 To 0', '0 To 10', '10 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 100', '100 To 110', '110 To 120']
PRCP_headings = ['-10 To 0', '0 To 10', '10 To 25', '25 To 32', '32 To 40', '40 To 55', '55 To 60', '60 To 65', '65 To 70', '70 To 75', '75 To 80', '80 To 85', '85 To 90', '90 To 100', '100 To 110', '110 To 120']

### TMAX

weather_data_TMAX_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TMAX'], bins)).rename_axis(columns=None, index=None)

weather_data_TMAX_df


,"(-50, -25]","(-25, -10]","(-10, 0]","(0, 10]","(10, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 100]","(100, 110]","(110, 120]"
CA001011500,0,1,1,3,3,1,2,11,8,9,13,16,13,10,11,29,19,17
CA001012055,0,2,10,1,4,6,4,14,12,3,11,2,11,4,11,21,10,7
CA001012475,0,0,0,2,0,2,1,4,3,8,4,17,22,13,22,30,32,13
CA001012710,0,0,0,2,0,1,2,5,3,6,7,14,10,15,19,39,23,17
CA001014820,1,1,0,5,1,2,13,26,11,14,17,19,14,13,7,16,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,3,5,40,15,12,15,16,25,8,6,4,6,6,10,5,19,20,18
USW00096406,20,20,11,6,12,1,6,4,1,1,3,1,5,3,4,10,5,8
USW00096407,25,10,19,9,12,2,3,8,2,5,1,1,1,4,3,2,8,10
USW00096408,22,17,6,13,12,4,10,20,2,4,5,4,1,3,3,7,5,11


In [6]:
# Sum all columns and rows TMAX
sums = weather_data_TMAX_df.sum().sum()
print(sums)

6829854


In [7]:
# Write TMAX weather data to parquet file

counter =-1
for h in headings:
    counter += 1 
    weather_data_TMAX_df.rename(columns={weather_data_TMAX_df.columns[counter]: h}, inplace = True)

# Converting  to string
for h in headings:
    weather_data_TMAX_df[h] = weather_data_TMAX_df[h].astype(str)

# Writing to parquet
weather_data_TMAX_df.to_parquet('result_files/us_ca_daily_TMAX_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

# Displaying the types of data after conversion
print("\nTypes of data after conversion:\n", weather_data_TMAX_df.dtypes)


Types of data after conversion:
 -50 To -25    object
-25 To -10    object
-10 To 0      object
0 To 10       object
10 To 25      object
25 To 32      object
32 To 40      object
40 To 55      object
55 To 60      object
60 To 65      object
65 To 70      object
70 To 75      object
75 To 80      object
80 To 85      object
85 To 90      object
90 To 100     object
100 To 110    object
110 To 120    object
dtype: object


In [8]:
# Read TMAX weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_TMAX_df = pd.read_parquet('result_files/us_ca_daily_TMAX_weather.parquet.gzip', engine="fastparquet") 
weather_data_TMAX_df

,-50 To -25,-25 To -10,-10 To 0,0 To 10,10 To 25,25 To 32,32 To 40,40 To 55,55 To 60,60 To 65,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 100,100 To 110,110 To 120
index,,,,,,,,,,,,,,,,,,
CA001011500,0,1,1,3,3,1,2,11,8,9,13,16,13,10,11,29,19,17
CA001012055,0,2,10,1,4,6,4,14,12,3,11,2,11,4,11,21,10,7
CA001012475,0,0,0,2,0,2,1,4,3,8,4,17,22,13,22,30,32,13
CA001012710,0,0,0,2,0,1,2,5,3,6,7,14,10,15,19,39,23,17
CA001014820,1,1,0,5,1,2,13,26,11,14,17,19,14,13,7,16,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,3,5,40,15,12,15,16,25,8,6,4,6,6,10,5,19,20,18
USW00096406,20,20,11,6,12,1,6,4,1,1,3,1,5,3,4,10,5,8
USW00096407,25,10,19,9,12,2,3,8,2,5,1,1,1,4,3,2,8,10


In [9]:
### TMIN 

weather_data_TMIN_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TMIN'], bins)).rename_axis(columns=None, index=None)

weather_data_TMIN_df

,"(-50, -25]","(-25, -10]","(-10, 0]","(0, 10]","(10, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 100]","(100, 110]","(110, 120]"
CA001011500,3,5,6,25,33,17,20,36,13,7,3,13,6,10,10,15,21,19
CA001012055,4,10,17,13,7,15,13,27,9,1,5,4,6,0,8,12,7,5
CA001012475,2,1,1,2,7,7,12,48,15,17,18,23,12,11,14,35,39,58
CA001012710,3,2,2,8,20,12,28,40,7,19,15,12,5,9,14,25,29,27
CA001014820,3,6,12,36,40,18,21,28,5,7,8,12,7,10,9,18,17,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,28,25,62,10,23,8,6,31,7,2,5,6,9,9,10,23,19,4
USW00096406,21,11,7,5,6,6,6,5,6,6,4,4,4,4,2,11,23,13
USW00096407,9,16,9,8,11,1,8,9,3,5,5,2,2,8,3,14,7,10
USW00096408,18,5,10,16,11,5,6,12,9,6,3,4,12,6,6,12,8,7


In [10]:
# Sum all columns and rows TMIN
sums = weather_data_TMIN_df.sum().sum()
print(sums)

7520687


In [11]:
# Write TMIN weather data to parquet file

counter =-1
for h in headings:
    counter += 1 
    weather_data_TMIN_df.rename(columns={weather_data_TMIN_df.columns[counter]: h}, inplace = True)

# Converting  to string
for h in headings:
    weather_data_TMIN_df[h] = weather_data_TMIN_df[h].astype(str)

# Writing to parquet
weather_data_TMIN_df.to_parquet('result_files/us_ca_daily_TMIN_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

# Displaying the types of data after conversion
print("\nTypes of data after conversion:\n", weather_data_TMIN_df.dtypes)


Types of data after conversion:
 -50 To -25    object
-25 To -10    object
-10 To 0      object
0 To 10       object
10 To 25      object
25 To 32      object
32 To 40      object
40 To 55      object
55 To 60      object
60 To 65      object
65 To 70      object
70 To 75      object
75 To 80      object
80 To 85      object
85 To 90      object
90 To 100     object
100 To 110    object
110 To 120    object
dtype: object


In [12]:
# Read TMIN weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_TMIN_df = pd.read_parquet('result_files/us_ca_daily_TMIN_weather.parquet.gzip', engine="fastparquet") 
weather_data_TMIN_df

,-50 To -25,-25 To -10,-10 To 0,0 To 10,10 To 25,25 To 32,32 To 40,40 To 55,55 To 60,60 To 65,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 100,100 To 110,110 To 120
index,,,,,,,,,,,,,,,,,,
CA001011500,3,5,6,25,33,17,20,36,13,7,3,13,6,10,10,15,21,19
CA001012055,4,10,17,13,7,15,13,27,9,1,5,4,6,0,8,12,7,5
CA001012475,2,1,1,2,7,7,12,48,15,17,18,23,12,11,14,35,39,58
CA001012710,3,2,2,8,20,12,28,40,7,19,15,12,5,9,14,25,29,27
CA001014820,3,6,12,36,40,18,21,28,5,7,8,12,7,10,9,18,17,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,28,25,62,10,23,8,6,31,7,2,5,6,9,9,10,23,19,4
USW00096406,21,11,7,5,6,6,6,5,6,6,4,4,4,4,2,11,23,13
USW00096407,9,16,9,8,11,1,8,9,3,5,5,2,2,8,3,14,7,10


In [13]:
### TAVG

weather_data_TAVG_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['TAVG'], bins)).rename_axis(columns=None, index=None)

weather_data_TAVG_df

,"(-50, -25]","(-25, -10]","(-10, 0]","(0, 10]","(10, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 100]","(100, 110]","(110, 120]"
CA001011500,1,3,3,2,4,4,15,43,20,12,13,17,9,9,9,12,7,14
CA001012055,0,3,10,2,7,7,15,34,6,8,5,5,9,1,8,18,5,6
CA001012475,0,2,0,1,3,2,3,17,13,20,16,17,23,21,10,23,16,24
CA001012710,0,1,1,3,2,4,4,31,16,16,15,18,20,12,8,17,17,18
CA001014820,3,3,2,2,17,17,31,45,15,9,8,8,8,4,6,8,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,16,16,49,17,28,11,8,19,9,6,9,13,9,12,4,18,15,20
USW00096406,25,7,6,3,10,4,4,8,7,5,2,3,4,2,4,5,4,10
USW00096407,23,10,2,9,8,4,6,11,0,5,5,2,5,3,4,5,8,15
USW00096408,20,12,10,11,14,4,4,8,3,5,7,3,2,7,4,10,10,13


In [14]:
# Sum all columns and rows TAVG
sums = weather_data_TAVG_df.sum().sum()
print(sums)

7180303


In [15]:
# Write TAVG weather data to parquet file

counter =-1
for h in headings:
    counter += 1 
    weather_data_TAVG_df.rename(columns={weather_data_TAVG_df.columns[counter]: h}, inplace = True)

# Converting  to string
for h in headings:
    weather_data_TAVG_df[h] = weather_data_TAVG_df[h].astype(str)

# Writing to parquet
weather_data_TAVG_df.to_parquet('result_files/us_ca_daily_TAVG_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

# Displaying the types of data after conversion
print("\nTypes of data after conversion:\n", weather_data_TAVG_df.dtypes)


Types of data after conversion:
 -50 To -25    object
-25 To -10    object
-10 To 0      object
0 To 10       object
10 To 25      object
25 To 32      object
32 To 40      object
40 To 55      object
55 To 60      object
60 To 65      object
65 To 70      object
70 To 75      object
75 To 80      object
80 To 85      object
85 To 90      object
90 To 100     object
100 To 110    object
110 To 120    object
dtype: object


In [16]:
# Read TAVG weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_TAVG_df = pd.read_parquet('result_files/us_ca_daily_TAVG_weather.parquet.gzip', engine="fastparquet") 
weather_data_TAVG_df

,-50 To -25,-25 To -10,-10 To 0,0 To 10,10 To 25,25 To 32,32 To 40,40 To 55,55 To 60,60 To 65,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 100,100 To 110,110 To 120
index,,,,,,,,,,,,,,,,,,
CA001011500,1,3,3,2,4,4,15,43,20,12,13,17,9,9,9,12,7,14
CA001012055,0,3,10,2,7,7,15,34,6,8,5,5,9,1,8,18,5,6
CA001012475,0,2,0,1,3,2,3,17,13,20,16,17,23,21,10,23,16,24
CA001012710,0,1,1,3,2,4,4,31,16,16,15,18,20,12,8,17,17,18
CA001014820,3,3,2,2,17,17,31,45,15,9,8,8,8,4,6,8,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,16,16,49,17,28,11,8,19,9,6,9,13,9,12,4,18,15,20
USW00096406,25,7,6,3,10,4,4,8,7,5,2,3,4,2,4,5,4,10
USW00096407,23,10,2,9,8,4,6,11,0,5,5,2,5,3,4,5,8,15


In [17]:
### PRCP

weather_data_PRCP_df = pd.crosstab(weather_data_df['ID'], pd.cut(weather_data_df['PRCP'], bins)).rename_axis(columns=None, index=None)

weather_data_PRCP_df

,"(-10, 0]","(0, 10]","(10, 25]","(25, 32]","(32, 40]","(40, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]","(80, 85]","(85, 90]","(90, 100]","(100, 110]","(110, 120]"
CA001011500,185,45,24,8,6,12,4,4,4,5,0,1,6,6,5,2
CA001012055,166,18,9,4,8,6,3,0,3,0,3,0,2,2,4,4
CA001012475,364,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CA001012710,204,74,35,6,7,7,2,0,2,0,1,0,2,0,2,0
CA001014820,208,51,35,13,7,5,8,2,1,2,1,0,6,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,104,43,32,10,20,15,4,5,1,5,6,4,2,7,7,3
USW00096406,195,73,37,14,8,13,2,1,3,5,3,0,1,1,4,0
USW00096407,188,46,33,5,9,8,3,2,3,1,1,2,0,5,3,1
USW00096408,198,49,46,14,17,7,2,3,3,3,0,5,1,4,2,1


In [18]:
# Sum all columns and rows PRCP
sums = weather_data_PRCP_df.sum().sum()
print(sums)

8277010


In [19]:
# Write PRCP weather data to parquet file

counter =-1
for h in PRCP_headings:
    counter += 1 
    weather_data_PRCP_df.rename(columns={weather_data_PRCP_df.columns[counter]: h}, inplace = True)

# Converting  to string

for h in PRCP_headings:
    weather_data_PRCP_df[h] = weather_data_PRCP_df[h].astype(str)
    
# Writing to parquet
weather_data_PRCP_df.to_parquet('result_files/us_ca_daily_PRCP_weather.parquet.gzip', compression='gzip', engine="fastparquet")  

# Displaying the types of data after conversion
print("\nTypes of data after conversion:\n", weather_data_PRCP_df.dtypes)



Types of data after conversion:
 -10 To 0      object
0 To 10       object
10 To 25      object
25 To 32      object
32 To 40      object
40 To 55      object
55 To 60      object
60 To 65      object
65 To 70      object
70 To 75      object
75 To 80      object
80 To 85      object
85 To 90      object
90 To 100     object
100 To 110    object
110 To 120    object
dtype: object


In [20]:
# Read PRCP weather paqquet file into a pandas dataframe to verify data integrity is maintained
weather_data_PRCP_df = pd.read_parquet('result_files/us_ca_daily_PRCP_weather.parquet.gzip', engine="fastparquet") 
weather_data_PRCP_df


,-10 To 0,0 To 10,10 To 25,25 To 32,32 To 40,40 To 55,55 To 60,60 To 65,65 To 70,70 To 75,75 To 80,80 To 85,85 To 90,90 To 100,100 To 110,110 To 120
index,,,,,,,,,,,,,,,,
CA001011500,185,45,24,8,6,12,4,4,4,5,0,1,6,6,5,2
CA001012055,166,18,9,4,8,6,3,0,3,0,3,0,2,2,4,4
CA001012475,364,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CA001012710,204,74,35,6,7,7,2,0,2,0,1,0,2,0,2,0
CA001014820,208,51,35,13,7,5,8,2,1,2,1,0,6,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USW00096405,104,43,32,10,20,15,4,5,1,5,6,4,2,7,7,3
USW00096406,195,73,37,14,8,13,2,1,3,5,3,0,1,1,4,0
USW00096407,188,46,33,5,9,8,3,2,3,1,1,2,0,5,3,1
